# 3. Twitter analysis using SVMs

In [20]:
"""
Author      : Yi-Chieh Wu, Sriram Sankararman
Description : Twitter
"""

from string import punctuation

import numpy as np

# !!! MAKE SURE TO USE SVC.decision_function(X), NOT SVC.predict(X) !!!
# (this makes ``continuous-valued'' predictions)
from sklearn.svm import SVC
from sklearn.cross_validation import StratifiedKFold
from sklearn import metrics

######################################################################
# functions -- input/output
######################################################################

def read_vector_file(fname):
    """
    Reads and returns a vector from a file.
    
    Parameters
    --------------------
        fname  -- string, filename
        
    Returns
    --------------------
        labels -- numpy array of shape (n,)
                    n is the number of non-blank lines in the text file
    """
    return np.genfromtxt(fname)


def write_label_answer(vec, outfile):
    """
    Writes your label vector to the given file.
    
    Parameters
    --------------------
        vec     -- numpy array of shape (n,) or (n,1), predicted scores
        outfile -- string, output filename
    """
    
    # for this project, you should predict 70 labels
    if(vec.shape[0] != 70):
        print("Error - output vector should have 70 rows.")
        print("Aborting write.")
        return
    
    np.savetxt(outfile, vec)    


######################################################################
# functions -- feature extraction
######################################################################

def extract_words(input_string):
    """
    Processes the input_string, separating it into "words" based on the presence
    of spaces, and separating punctuation marks into their own words.
    
    Parameters
    --------------------
        input_string -- string of characters
    
    Returns
    --------------------
        words        -- list of lowercase "words"
    """
    
    for c in punctuation :
        input_string = input_string.replace(c, ' ' + c + ' ')
    return input_string.lower().split()


def extract_dictionary(infile):
    """
    Given a filename, reads the text file and builds a dictionary of unique
    words/punctuations.
    
    Parameters
    --------------------
        infile    -- string, filename
    
    Returns
    --------------------
        word_list -- dictionary, (key, value) pairs are (word, index)
    """
    
    word_list = {}
    with open(infile, 'rU') as fid :
        ### ========== TODO : START ========== ###
        # part 1a: process each line to populate word_list
        index = 0
        lines = fid.readlines()
        for line in lines:
            words = extract_words(line)
            for word in words:
                if word not in word_list.keys():
                    word_list[word] = index
                    index += 1
        ### ========== TODO : END ========== ###

    return word_list


def extract_feature_vectors(infile, word_list):
    """
    Produces a bag-of-words representation of a text file specified by the
    filename infile based on the dictionary word_list.
    
    Parameters
    --------------------
        infile         -- string, filename
        word_list      -- dictionary, (key, value) pairs are (word, index)
    
    Returns
    --------------------
        feature_matrix -- numpy array of shape (n,d)
                          boolean (0,1) array indicating word presence in a string
                            n is the number of non-blank lines in the text file
                            d is the number of unique words in the text file
    """
    
    num_lines = sum(1 for line in open(infile,'rU'))
    num_words = len(word_list)
    feature_matrix = np.zeros((num_lines, num_words))
    
    with open(infile, 'rU') as fid :
        ### ========== TODO : START ========== ###
        # part 1b: process each line to populate feature_matrix
        lines = fid.readlines()
        for i in range(len(lines)):
            words = extract_words(lines[i])
            for j in range(len(words)):
                feature_matrix[i][word_list[words[j]]] = 1
        ### ========== TODO : END ========== ###
        
    return feature_matrix


######################################################################
# functions -- evaluation
######################################################################

def performance(y_true, y_pred, metric="accuracy"):
    """
    Calculates the performance metric based on the agreement between the 
    true labels and the predicted labels.
    
    Parameters
    --------------------
        y_true -- numpy array of shape (n,), known labels
        y_pred -- numpy array of shape (n,), (continuous-valued) predictions
        metric -- string, option used to select the performance measure
                  options: 'accuracy', 'f1-score', 'auroc', 'precision',
                           'sensitivity', 'specificity'        
    
    Returns
    --------------------
        score  -- float, performance score
    """
    # map continuous-valued predictions to binary labels
    y_label = np.sign(y_pred)
    y_label[y_label==0] = 1
    
    ### ========== TODO : START ========== ###
    # part 2a: compute classifier performance
    if metric == 'accuracy':
        return metrics.accuracy_score(y_true, y_label)
    elif metric == 'f1-score':
        return metrics.f1_score(y_true, y_label)
    elif metric == 'auroc':
        return metrics.roc_auc_score(y_true, y_label)
    elif metric == 'precision':
        return metrics.precision_score(y_true, y_label)
    elif metric == 'sensitivity':
        tn, fp, fn, tp = metrics.confusion_matrix(y_true, y_label).ravel()
        return tp / float(tp + fn)
    elif metric == 'specificity':
        tn, fp, fn, tp = metrics.confusion_matrix(y_true, y_label).ravel()
        return tn / float(fp + tn)
    else:
        raise ValueError('invliad metric {}'.format(metric))

    ### ========== TODO : END ========== ###


def cv_performance(clf, X, y, kf, metric="accuracy"):
    """
    Splits the data, X and y, into k-folds and runs k-fold cross-validation.
    Trains classifier on k-1 folds and tests on the remaining fold.
    Calculates the k-fold cross-validation performance metric for classifier
    by averaging the performance across folds.
    
    Parameters
    --------------------
        clf    -- classifier (instance of SVC)
        X      -- numpy array of shape (n,d), feature vectors
                    n = number of examples
                    d = number of features
        y      -- numpy array of shape (n,), binary labels {1,-1}
        kf     -- cross_validation.KFold or cross_validation.StratifiedKFold
        metric -- string, option used to select performance measure
    
    Returns
    --------------------
        score   -- float, average cross-validation performance across k folds
    """
    
    ### ========== TODO : START ========== ###
    # part 2b: compute average cross-validation performance
    kFold = kf(y, n_folds=5)
    sum_score = 0
    for train_index, test_index in kFold:
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        clf.fit(X_train, y_train)
        y_pred = clf.decision_function(X_test)
        sum_score += performance(y_test, y_pred, metric=metric)
    return sum_score / float(5)
    ### ========== TODO : END ========== ###


def select_param_linear(X, y, kf, metric="accuracy"):
    """
    Sweeps different settings for the hyperparameter of a linear-kernel SVM,
    calculating the k-fold CV performance for each setting, then selecting the
    hyperparameter that 'maximize' the average k-fold CV performance.
    
    Parameters
    --------------------
        X      -- numpy array of shape (n,d), feature vectors
                    n = number of examples
                    d = number of features
        y      -- numpy array of shape (n,), binary labels {1,-1}
        kf     -- cross_validation.KFold or cross_validation.StratifiedKFold
        metric -- string, option used to select performance measure
    
    Returns
    --------------------
        C -- float, optimal parameter value for linear-kernel SVM
    """
    
    print('Linear SVM Hyperparameter Selection based on ' + str(metric) + ':')
    C_range = 10.0 ** np.arange(-3, 3)
    
    ### ========== TODO : START ========== ###
    # part 2c: select optimal hyperparameter using cross-validation
    results = []
    for C in C_range:
        print('C: {}'.format(C))
        result = cv_performance(SVC(C=C, kernel='linear'), X, y, kf=kf, metric=metric)
        print('result: {0:.4f}\n'.format(result))
        results.append(result)
    return C_range[np.argmax(results)]
    ### ========== TODO : END ========== ###


def select_param_rbf(X, y, kf, metric="accuracy"):
    """
    Sweeps different settings for the hyperparameters of an RBF-kernel SVM,
    calculating the k-fold CV performance for each setting, then selecting the
    hyperparameters that 'maximize' the average k-fold CV performance.
    
    Parameters
    --------------------
        X       -- numpy array of shape (n,d), feature vectors
                     n = number of examples
                     d = number of features
        y       -- numpy array of shape (n,), binary labels {1,-1}
        kf     -- cross_validation.KFold or cross_validation.StratifiedKFold
        metric  -- string, option used to select performance measure
    
    Returns
    --------------------
        C, gamma,  -- tuple of floats, optimal parameter values for an RBF-kernel SVM
    """
    
    print('RBF SVM Hyperparameter Selection based on ' + str(metric) + ':')
    
    ### ========== TODO : START ========== ###
    # part 3b: create grid, then select optimal hyperparameters using cross-validation
    # C=100, gamma=0.01 seems good
    C_range = 10.0 ** np.arange(-3, 3)
    gamma_range = 10.0 ** np.arange(-3, 3)
    
    # C=70, gamma=0.004 seems good
    C_range = 100 + 10 * np.arange(-3, 3)
    gamma_range = 0.01 + 0.003 * np.arange(-3, 3)
    
    #C=80, gamma=0.005 seems good
    #C_range = 70 + 5 * np.arange(-3, 3)
    #gamma_range = 0.004 + 0.001 * np.arange(-3, 3)
    
    results = []
    for i in range(len(C_range)):
        C = C_range[i]
        for j in range(len(gamma_range)):
            gamma = gamma_range[j]
            print('C: {}'.format(C))
            print('gamma: {}'.format(gamma))
            result = cv_performance(SVC(C=C, kernel='rbf', gamma=gamma), X, y, kf=kf, metric=metric)
            print('result: {0:.4f}\n'.format(result))
            results.append(result)
        
    index = np.argmax(results)
    print(index, len(gamma_range))
    gamma_index = int(index % len(gamma_range))
    C_index = int(index / len(gamma_range))
    return C_range[C_index], gamma_range[gamma_index]
    ### ========== TODO : END ========== ###


def performance_test(clf, X, y, metric="accuracy"):
    """
    Estimates the performance of the classifier using the 95% CI.
    
    Parameters
    --------------------
        clf          -- classifier (instance of SVC)
                          [already fit to data]
        X            -- numpy array of shape (n,d), feature vectors of test set
                          n = number of examples
                          d = number of features
        y            -- numpy array of shape (n,), binary labels {1,-1} of test set
        metric       -- string, option used to select performance measure
    
    Returns
    --------------------
        score        -- float, classifier performance
    """

    ### ========== TODO : START ========== ###
    # part 4b: return performance on test data by first computing predictions and then calling performance
    y_pred = clf.predict(X)
#     y_pred = clf.decision_function(X)
#     y_label = np.sign(y_pred)
#     y_label[y_label==0] = 1
#     y_pred = y_label
    
    if metric == 'accuracy':
        return metrics.accuracy_score(y, y_pred)
    elif metric == 'f1-score':
        return metrics.f1_score(y, y_pred)
    elif metric == 'auroc':
        return metrics.roc_auc_score(y, y_pred)
    elif metric == 'precision':
        return metrics.precision_score(y, y_pred)
    elif metric == 'sensitivity':
        tn, fp, fn, tp = metrics.confusion_matrix(y, y_pred).ravel()
        return tp / float(tp + fn)
    elif metric == 'specificity':
        tn, fp, fn, tp = metrics.confusion_matrix(y, y_pred).ravel()
        return tn / float(fp + tn)
    else:
        raise ValueError('invliad metric {}'.format(metric))
        
    ### ========== TODO : END ========== ###

In [21]:
np.random.seed(1234)
    
# read the tweets and its labels   
dictionary = extract_dictionary('../data/tweets.txt')
X = extract_feature_vectors('../data/tweets.txt', dictionary)
y = read_vector_file('../data/labels.txt')
    
metric_list = ["accuracy", "f1_score", "auroc", "precision", "sensitivity", "specificity"]

/Users/one/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:93: DeprecationWarning: 'U' mode is deprecated
/Users/one/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:127: DeprecationWarning: 'U' mode is deprecated
/Users/one/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:131: DeprecationWarning: 'U' mode is deprecated


## 3.1 Feature Extraction

In [22]:
# part 1c: split data into training (training + cross-validation) and testing set
X_train, X_test, y_train, y_test = X[:560], X[560:], y[:560], y[560:]

In [23]:
X_train.shape

(560, 1811)

In [12]:
len(y_test)

70

In [13]:
np.sum(y == 1) / float(len(y))

0.66349206349206347

In [14]:
np.sum(y_train == 1) / float(len(y_train))

0.70892857142857146

In [15]:
np.sum(y_test == 1) / float(len(y_test))

0.29999999999999999

## 3.2 Hyperparameter Selection for a Linear-Kernel SVM

In [28]:
# part 2b: create stratified folds (5-fold CV)

**(b) Why is it beneficial to maintain class proportions across folds?**

Since we are trying to figure out the optimal hyperparameter for the training set, it makes sense to tune our hyperparameter with the folds that have same distribution as the original training set. Also, if we don’t maintain class proportions, some fold might consist of mostly positive samples, from which the model can’t really learn anything. 


**(c) finding the optimal C**

In [8]:
# part 2d: for each metric, select optimal hyperparameter for linear-kernel SVM using CV
options = ['accuracy', 'f1-score', 'auroc', 'precision', 'sensitivity', 'specificity']
for option in options:
    print('best C: {}\n'.format(select_param_linear(X_train, y_train, StratifiedKFold, metric=option)))

Linear SVM Hyperparameter Selection based on accuracy:
C: 0.001
result: 0.7089

C: 0.01
result: 0.7107

C: 0.1
result: 0.8060

C: 1.0
result: 0.8146

C: 10.0
result: 0.8182

C: 100.0
result: 0.8182

best C: 10.0

Linear SVM Hyperparameter Selection based on f1-score:
C: 0.001
result: 0.8297

C: 0.01
result: 0.8306

C: 0.1
result: 0.8755

C: 1.0
result: 0.8749

C: 10.0
result: 0.8766

C: 100.0
result: 0.8766

best C: 10.0

Linear SVM Hyperparameter Selection based on auroc:
C: 0.001
result: 0.5000

C: 0.01
result: 0.5031

C: 0.1
result: 0.7188

C: 1.0
result: 0.7531

C: 10.0
result: 0.7592

C: 100.0
result: 0.7592

best C: 10.0

Linear SVM Hyperparameter Selection based on precision:
C: 0.001
result: 0.7089

C: 0.01
result: 0.7102

C: 0.1
result: 0.8357

C: 1.0
result: 0.8562

C: 10.0
result: 0.8595

C: 100.0
result: 0.8595

best C: 10.0

Linear SVM Hyperparameter Selection based on sensitivity:
C: 0.001
result: 1.0000

C: 0.01
result: 1.0000

C: 0.1
result: 0.9294

C: 1.0
result: 0.901

**(d) How does the 5-fold CV performance vary with C and the performance metric?**

For accuracy, F1-score, AUROC, precision, and sensitivity, the performance increases as C increases.

For sensitivity, the performance decreases as C increases.

## 3.3 Hyperparameter Selection for an RBF-kernel SVM

**(a) Describe the role of the additional hyperparameter γ for an RBF-kernel SVM. How does γ affect generalization error?**

RBF-kernel measures the similarity between two data points. The hyperparameter γ represents the inverse of the variance of RBF-kernel. Small value of γ means large variance. In other words, even two points are not that close, they can be interpreted as close to each other. On the other hand, large value of γ means small variance. In other words, two points are considered close to each other if they are really close to each other. Therefore, the larger the γ is, the more it overfits the model because it uses only data points that are really close to each other. The smaller the γ is, the more it generalizes the model better.

**(b) Explain what kind of grid you used and why.**

First, I kept the same range of C from previous problem, and set the gamma to have the same range as C.

* C_range = 10.0 \*\* np.arange(-3, 3)
* gamma_range = 10.0 \*\* np.arange(-3, 3)

This gives me an insight that C=100 and gamma=0.01 are good values.

Next, I tried to refine C and gamma, and set the ranges as the follow.

* C_range = 100 + 10 * np.arange(-3, 3)
* gamma_range = 0.01 + 0.003 * np.arange(-3, 3)

It seems C=70 and gamma=0.004 are good.

In [24]:
# part 3c: for each metric, select optimal hyperparameter for RBF-SVM using CV
options = ['accuracy', 'f1-score', 'auroc', 'precision', 'sensitivity', 'specificity']
for option in options:
    print('best (C, gamma): {}\n'.format(select_param_rbf(X_train, y_train, StratifiedKFold, metric=option)))

RBF SVM Hyperparameter Selection based on accuracy:
C: 70
gamma: 0.0009999999999999992
result: 0.8112

C: 70
gamma: 0.004
result: 0.8164

C: 70
gamma: 0.007
result: 0.8200

C: 70
gamma: 0.01
result: 0.8165

C: 70
gamma: 0.013000000000000001
result: 0.8147

C: 70
gamma: 0.016
result: 0.8165

C: 80
gamma: 0.0009999999999999992
result: 0.8077

C: 80
gamma: 0.004
result: 0.8200

C: 80
gamma: 0.007
result: 0.8200

C: 80
gamma: 0.01
result: 0.8165

C: 80
gamma: 0.013000000000000001
result: 0.8147

C: 80
gamma: 0.016
result: 0.8165

C: 90
gamma: 0.0009999999999999992
result: 0.8095

C: 90
gamma: 0.004
result: 0.8182

C: 90
gamma: 0.007
result: 0.8200

C: 90
gamma: 0.01
result: 0.8165

C: 90
gamma: 0.013000000000000001
result: 0.8147

C: 90
gamma: 0.016
result: 0.8165

C: 100
gamma: 0.0009999999999999992
result: 0.8095

C: 100
gamma: 0.004
result: 0.8200

C: 100
gamma: 0.007
result: 0.8200

C: 100
gamma: 0.01
result: 0.8165

C: 100
gamma: 0.013000000000000001
result: 0.8147

C: 100
gamma: 0.01

**(c) report only the best score for each metric, along with the accompanying C and γ setting.**

**(d) How does the CV performance vary with the hyperparameters of the RBF-kernel SVM?**

The performance increases as C increases. The performance increases as gamma decreases.

## 3.4 Test Set Performance

**(a) explains your choice**

I chose C=10 for linear kernel because most of the metrics perform the best when C=10.

I chose C=80 and gamma=0.005 for rbf kernel because most of the metrics perform the best when C=80 and gamma=0.005.

In [25]:
# part 4a: train linear- and RBF-kernel SVMs with selected hyperparameters
print('training linear SVM...')
clf_linear = SVC(C=10, kernel='linear')
clf_linear.fit(X_train, y_train)

print('training RBF-kernel SVM...')
clf_rbf = SVC(C=70, gamma=0.004, kernel='rbf')
clf_rbf.fit(X_train, y_train)

training linear SVM...
training RBF-kernel SVM...


SVC(C=70, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.004, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [26]:
# part 4c: report performance on test data
options = ['accuracy', 'f1-score', 'auroc', 'precision', 'sensitivity', 'specificity']
print('Testing linear SVM...')
for option in options:
    print('{0}: {1:.4f}'.format(option, performance_test(clf_linear, X_test, y_test, metric=option)))

print('\nTesting RBF-kernel SVM...')
for option in options:
    print('{0}: {1:.4f}'.format(option, performance_test(clf_rbf, X_test, y_test, metric=option)))

Testing linear SVM...
accuracy: 0.7429
f1-score: 0.4375
auroc: 0.6259
precision: 0.6364
sensitivity: 0.3333
specificity: 0.9184

Testing RBF-kernel SVM...
accuracy: 0.7571
f1-score: 0.4516
auroc: 0.6361
precision: 0.7000
sensitivity: 0.3333
specificity: 0.9388


**(c) How do the test performance of your two classifiers compare?**

Testing linear SVM...
* accuracy: 0.7429
* f1-score: 0.4375
* auroc: 0.6259
* precision: 0.6364
* sensitivity: 0.3333
* specificity: 0.9184

Testing RBF-kernel SVM...
* accuracy: 0.7571
* f1-score: 0.4516
* auroc: 0.6361
* precision: 0.7000
* sensitivity: 0.3333
* specificity: 0.9388

It seems both SVMs have the same performance. My guess is that since the shape of X_train is (560, 1811), which means that number of features is much larger than number of training examples, increasing the complexity of the model does not help the performance. But in general, where there are way more training examples than features, RBF-kernel should perform better because it allows the model to be more complex than linear kernel.